In [2]:
import numpy as np
import pandas as pd

In [6]:
# CONFIG

SAMPLING_RATE = 44100

SPLIT_IN_SECS = 7  # in seconds
CHUNK_MIN_SIZE = 1  # in seconds



#----- BEST MODEL ---------
#7 sec, sr=44100, lr=2e-05, num_mel_bins=178
model_name = 'sm-training-custom-2023-07-16-14-12-31-154'
checkpoint = 'checkpoint-13200'

#model_name = 'sm-training-custom-2023-06-21-12-27-40-579'
#checkpoint = 'checkpoint-3504'


# Wojtek
#model_name = 'sm-training-custom-2023-07-13-21-52-21-077'
#checkpoint = 'checkpoint-8934'

# Janek
#model_name = 'sm-training-custom-2023-07-13-09-09-10-608'
#checkpoint = 'checkpoint-11277'

FOLDER_TO_PROCESS = 'test'

In [7]:
df = pd.read_csv(f"/root/data/experiments/exp_2023-07-13_chunk_fixed_predictions/predictions_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs_{SAMPLING_RATE}.csv")
#df['logits_str'] = df['logits']
df['logits'] = df['logits'].apply(lambda x: np.array([float(i) for i in x.strip('[').strip(']').replace('n', '').split(' ') if i != '']))
df['probs'] = df['logits'].apply(lambda x: np.array(np.exp(x) / np.sum(np.exp(x))))

df.head()

,file_name,predicted_class_id,logits,probs
0,0.wav,14,"[-0.0636066198, -0.548345983, 0.395147562, -0....","[6.512336759480739e-07, 4.010672679876675e-07,..."
1,0.wav,14,"[-0.060945541, -0.59027719, 0.40035322, -0.047...","[6.168012928774656e-07, 3.6329503376821374e-07..."
2,1.wav,60,"[-1.1129682, -1.5018668, 0.13556546, -0.916900...","[2.3797063602979273e-07, 1.6129721000230855e-0..."
3,10.wav,26,"[-0.52893174, 1.666247, -0.8008976, 0.18271926...","[1.0608557382090603e-07, 9.528188566480471e-07..."
4,10.wav,26,"[0.1385512, -1.4241198, -1.9842126, -0.7965207...","[0.0005282360721187519, 0.00011070536358422602..."


In [8]:
# logits
voting_method = 'logits'
p = df.groupby('file_name').logits.sum().apply(lambda x: np.argmax(x)).reset_index()
p.columns = ['file_name', 'predicted_class_id']
#p.to_csv(f"/root/data/experiments/models/{model_name}/{checkpoint}/RESULTS_predictions_{voting_method}_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs.csv", index=False)
p.to_csv(f"/root/data/experiments/exp_2023-07-13_chunk_fixed_predictions/RESULTS_predictions_{voting_method}_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs_{SAMPLING_RATE}.csv", index=False)


# probs
voting_method = 'probs'
p = df.groupby('file_name').probs.sum().apply(lambda x: np.argmax(x)).reset_index()
p.columns = ['file_name', 'predicted_class_id']
#p.to_csv(f"/root/data/experiments/models/{model_name}/{checkpoint}/RESULTS_predictions_{voting_method}_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs.csv", index=False)
p.to_csv(f"/root/data/experiments/exp_2023-07-13_chunk_fixed_predictions/RESULTS_predictions_{voting_method}_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs_{SAMPLING_RATE}.csv", index=False)


# voting
voting_method = 'voting'
unique_files = sorted(df['file_name'].unique(), key=lambda x: int(x.strip('.wav')))

num = 0
results = []
for file in unique_files:
    temp_df = df[df['file_name'] == file]
    best_class_id = temp_df['predicted_class_id'].value_counts().idxmax()
    results.append([file, best_class_id])
    num += 1

p = pd.DataFrame(results)
p.columns = ['file_name', 'predicted_class_id']

#p.to_csv(f"/root/data/experiments/models/{model_name}/{checkpoint}/RESULTS_predictions_{voting_method}_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs.csv", index=False)
p.to_csv(f"/root/data/experiments/exp_2023-07-13_chunk_fixed_predictions/RESULTS_predictions_{voting_method}_{model_name}_{checkpoint}_{FOLDER_TO_PROCESS}_{SPLIT_IN_SECS}_secs_{SAMPLING_RATE}.csv", index=False)

In [ ]:
Tutorial5-2023-06-21-12-27-40-579